
# Introduction to AlTar/Pyre applications


### 1. Introduction

An AlTar application is based on the [pyre](https://github.com/pyre/pyre) framework. Compared with traditional Python programming, the `pyre` framework provides enhanced features for developing high performance scientific applications, including

- It introduces configurable components. Users may want to choose between different algorithms/methods for a given procedure, which in general requires developing new applications and/or recompiling the source code. With configurable components, users can now specify their choices at runtime with the same application.  

- It offers users an easy way to configure parameters/settings in an application. To pass parameters through command line (e.g, by `argparse`) and property `setter` is usually a formidable task for applications with a large parameter set. In pyre, this can be done by one `json`-type configuration file.

- An AlTar/pyre application can deploy itself automatically to different computing platforms, such as a standalone computer, GPU workstations, computer clusters or clouds, with a simple change of the `shell` configuration. 

- Pyre also integrates supports to [GNU Scientific Library](https://www.gnu.org/software/gsl/) (for linear algebra and statistics), and [CUDA](https://developer.nvidia.com/cuda-downloads) (for GPU accelerated computing), to achieve high performance while maintaining the user-friendly interface offered by Python. It also offers an easy procedure for users to develop their own applications with mixed Python/C/C++/Fortran/CUDA programming. 

To achieve above goals, An AlTar/pyre application is constructed and runs slightly different from regular Python script applications. In this tutorial, we will use a `Hello world!` application to demonstrate the structure of an AlTar application.

### 2. The Hello application

We create below an application to say "Hello" to someone (parameter `who`)  several times (parameter `times`).

In [1]:
# import the altar module
import altar

# create an application based on altar.application
class HelloApp(altar.application, family='altar.applications.hello'):
    """
    A specialized AlTar application to say hello 
    """
    
    # user configurable parameters/components
    who = altar.properties.str(default='world')
    who.doc = "the person to say hello to"
    
    times = altar.properties.int(default=1)
    times.validators = altar.constraints.isPositive()
    times.doc = "how many times you want to say hello"
    
    # define methods
    def main(self):
        """
        The main method
        """
        for i in range(self.times):
            print(f"Hello {self.who}!")
        # all done
        return

The `HelloApp` application is derived from the `altar.application` base class in order to inherit various features offered by the pyre framework. It has two configurable components, `who` and `times`. The components can be basic Python data types, specified by altar.properties.[int, float, str, list, dict ...], or user developed class .

To run the HelloApp, we create an instance with a name='hello'. We pass the settings of `who` and `times` by a configuration file [hello.pfg](hello.pfg) (by default, the instance searches for a `NAME.pfg` configuration file with `NAME` the same as the instance name): 

```
; application instance name
hello:
    ; components configuration
    who = AlTar users ; no start/end quotes for strings are needed in pfg file
    times = 3
```
In a `pfg` (pyre-config) configuration file, indents are used to show hierarchy of each configurable components. An alternative is to use Python style, e.g., 

```
; an alternative way to write configurations
hello.who = AlTar users
hello.times = 3
```


In [2]:
# create a HelloApp instance with a name
helloapp = HelloApp(name='hello')
# when it is created, it searches for settings in hello.pfg, if present, to initialize configurable components

# run the instance main method
helloapp.main()

Hello AlTar users!
Hello AlTar users!
Hello AlTar users!


Once an instance is created(registered), all its components are processed to be regular Python objects which you may access/modify.

In [3]:
print(f"'{helloapp.who}' is going to be changed")
helloapp.who='pyre users'
helloapp.main()

'AlTar users' is going to be changed
Hello pyre users!
Hello pyre users!
Hello pyre users!


You may also modify the [hello.pfg](hello.pfg) file for new configurations and re-run the program. Caveat: for jupyter/ipython, you may need to restart the kernel for new settings to be accepted.

### 3. Run HelloApp from command line

AlTar/pyre applications are designed to run as regular shell applications, which offer more options to run with command line arguments.

In [4]:
# run hello app from a shell with cmdLine settings
!python3 hello.py --who="World" --times=1

Hello World!


In [5]:
# run hello app with a specified configuration file
# hello2.pfg 
# ; application instance name
# hello:
##    ; configurations
##    who = pyre users
##    times = 1

!python3 hello.py --config=hello2.pfg

Hello pyre users!


In [6]:
# run hello app with a mixed configuration file and cmdLine settings
# pfg file settings will be overriden by the cmdLine ones
!python3 hello.py --config=hello2.pfg --times=2

Hello pyre users!
Hello pyre users!
